In [4]:
#Imports
from tslearn.clustering import TimeSeriesKMeans
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
from dtaidistance import dtw
from sklearn.metrics import davies_bouldin_score, silhouette_score

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

from sklearn.preprocessing import MinMaxScaler

In [1]:
#Get data 
cwd = os.path.normpath(os.path.dirname(os.getcwd()))
df = pd.read_csv(cwd+'/data/2feature_engineering_data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)
df.head()

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\rs1044\Documents\GitHub\Attacks-and-Security-of-Federated-Learning\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


,User1,User10,User11,User12,User13,User14,User15,User16,User17,User18,...,User27_lag_24hrs,User28_lag_24hrs,User29_lag_24hrs,User30_lag_24hrs,User31_lag_24hrs,User32_lag_24hrs,User33_lag_24hrs,User34_lag_24hrs,User35_lag_24hrs,User36_lag_24hrs
Date,,,,,,,,,,,,,,,,,,,,,
2012-07-08 00:00:00,0.200,0.099,0.312,0.152,1.227,0.162,0.089,1.079,0.366,0.063,...,0.040,0.100,0.522,0.058,0.211,0.066,0.627,0.844,0.413,0.117
2012-07-08 01:00:00,0.253,0.060,0.263,0.212,0.132,0.204,0.072,0.714,0.134,0.063,...,0.050,0.131,0.104,0.031,0.271,0.112,0.417,0.350,0.310,0.552
2012-07-08 02:00:00,0.441,0.028,0.257,0.218,0.144,0.193,0.067,0.706,0.127,0.061,...,0.051,0.106,0.163,0.022,0.280,0.121,0.324,0.313,0.285,0.055
2012-07-08 03:00:00,0.220,0.054,0.243,0.225,0.139,0.170,0.045,0.796,0.134,0.061,...,0.111,0.100,0.115,0.032,0.279,0.119,0.306,0.331,0.270,0.067
2012-07-08 04:00:00,0.635,0.060,0.245,0.188,1.157,0.146,0.064,0.755,0.154,0.063,...,0.083,0.094,0.148,0.067,0.184,0.120,0.287,0.275,0.273,0.033


In [2]:
# Only consider first 30 users
columns_to_keep = []

for column_name in df.columns:
    if column_name.startswith('User') and column_name[4:].isdigit():
        user_number = int(column_name[4:])
        if 1 <= user_number <= 30:
            columns_to_keep.append(column_name)

filtered_df = df[columns_to_keep]
filtered_df.reset_index(inplace=True)
filtered_df

,Date,User1,User10,User11,User12,User13,User14,User15,User16,User17,...,User28,User29,User3,User30,User4,User5,User6,User7,User8,User9
0,2012-07-08 00:00:00,0.200,0.099,0.312,0.152,1.227,0.162,0.089,1.079,0.366,...,0.144,0.418,0.979,0.024,0.092,0.084,0.509,0.106,0.133,0.011
1,2012-07-08 01:00:00,0.253,0.060,0.263,0.212,0.132,0.204,0.072,0.714,0.134,...,0.125,0.065,0.049,0.060,0.090,0.091,0.489,0.067,0.076,0.011
2,2012-07-08 02:00:00,0.441,0.028,0.257,0.218,0.144,0.193,0.067,0.706,0.127,...,0.088,0.048,0.046,0.057,0.091,0.076,0.573,0.066,0.071,0.015
3,2012-07-08 03:00:00,0.220,0.054,0.243,0.225,0.139,0.170,0.045,0.796,0.134,...,0.100,0.049,0.053,0.050,0.098,0.079,0.363,0.084,0.073,0.011
4,2012-07-08 04:00:00,0.635,0.060,0.245,0.188,1.157,0.146,0.064,0.755,0.154,...,0.088,0.066,0.040,0.022,0.096,0.133,0.595,0.129,0.073,0.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8587,2013-06-30 19:00:00,0.335,0.198,1.362,1.385,0.237,0.331,0.082,0.671,0.418,...,1.381,0.582,0.052,0.350,0.280,2.152,1.018,0.066,0.563,0.089
8588,2013-06-30 20:00:00,0.213,0.873,0.687,1.347,0.282,0.940,0.095,0.507,0.510,...,0.225,0.389,0.050,0.591,0.273,1.993,0.910,0.103,0.104,0.053
8589,2013-06-30 21:00:00,0.190,0.113,0.675,1.615,1.193,0.290,0.115,0.219,0.487,...,0.931,0.749,0.112,0.513,0.253,0.814,1.011,0.076,0.086,0.048
8590,2013-06-30 22:00:00,0.274,0.092,0.747,1.636,0.132,0.265,0.080,0.105,0.197,...,0.119,0.391,0.056,0.168,0.134,0.390,1.063,0.084,0.203,0.072


# K-manes Clustering

In [5]:
# Additional function to compute DTW Within-Cluster Variance
def compute_dtw_within_cluster_variance(data, labels, centroids):
    clusters_dtw_variance = []
    for cluster_id in set(labels):
        cluster_indices = np.where(labels == cluster_id)[0]
        cluster_data = data[cluster_indices]
        centroid = centroids[cluster_id]
        dtw_distances = [dtw.distance(cluster_data[i], centroid) for i in range(len(cluster_data))]
        clusters_dtw_variance.append(np.mean(dtw_distances))
    return np.mean(clusters_dtw_variance)

In [6]:
#Find best cluster size

metric = "dtw" # "euclidean", "dtw", "softdtw"

#Consider only 1 month, as dtw takes to long otherwise
filtered_df = filtered_df[(filtered_df['Date'].dt.month == 3) & (filtered_df['Date'].dt.year == 2013)]
# Drop date: Focus on Patterns, Not Specific Times / Avoiding Skewed Clusters / Numerical data
data_array = np.array(filtered_df.T.drop('Date').values)
# Scaling - non Gaussian data distributions, no outliers -> use Min-Max not MeanVariance scaling
data_array = MinMaxScaler().fit_transform(data_array)
# Reshape the data array to 2D
data_array_2d = data_array.reshape(data_array.shape[0], -1)

# Specify the maximum number of clusters you want to consider
max_clusters = 11
# Initialize variables to store best parameters
best_cluster_count = 0
best_labels = None
best_db_score = float('inf')  # Davies-Bouldin score
best_silhouette_score = -1    # Silhouette score
best_dtw_variance = float('inf')

# Iterate over possible cluster counts
for cluster_count in range(2, max_clusters + 1):
    kmeans = TimeSeriesKMeans(n_clusters=cluster_count, verbose=False, random_state=42, metric=metric)
    labels = kmeans.fit_predict(data_array)
    np.savetxt(f'../evaluations/clusters_KMeans{cluster_count}_dtw.csv', labels, delimiter=",")
    centroids = kmeans.cluster_centers_

    # Compute DTW Within-Cluster Variance
    dtw_variance = compute_dtw_within_cluster_variance(data_array, labels, centroids)

    # Evaluate the clustering performance using all scores
    db_score = davies_bouldin_score(data_array, labels)
    sil_score = silhouette_score(data_array, labels)
    print(f"Cluster Count: {cluster_count}, Davies-Bouldin Score: {db_score}, Silhouette Score: {sil_score}, DTW Variance: {dtw_variance}")

    # Update best parameters if the current scores and variance are better
    if db_score < best_db_score and sil_score > best_silhouette_score and dtw_variance < best_dtw_variance:
        best_db_score = db_score
        best_silhouette_score = sil_score
        best_dtw_variance = dtw_variance
        best_cluster_count = cluster_count
        best_labels = labels

# Use the best parameters for final clustering
final_kmeans = TimeSeriesKMeans(n_clusters=best_cluster_count, verbose=False, random_state=42, metric=metric)
final_labels = final_kmeans.fit_predict(data_array)

print("Final labels: ", final_labels)
print("Best cluster count: ", best_cluster_count)
print("Best Davies-Bouldin Score: ", best_db_score)
print("Best Silhouette Score: ", best_silhouette_score)
print("Best DTW Variance: ", best_dtw_variance)

dtw_variance = [3.0044869193855828, 2.5133436056132905, 2.452716918170206,1.854738022617764, 1.8828731296011114, 1.5463779597683391,
                ]

Cluster Count: 2, Davies-Bouldin Score: 1.5928038465603438, Silhouette Score: 0.05097102353245846, DTW Variance: 2.4745157189459968
Cluster Count: 3, Davies-Bouldin Score: 2.0129724082279696, Silhouette Score: 0.04325903141222982, DTW Variance: 2.48584328731337
Cluster Count: 4, Davies-Bouldin Score: 2.0031222953771937, Silhouette Score: 0.04205079597974595, DTW Variance: 2.3914854786496025
Cluster Count: 5, Davies-Bouldin Score: 2.157812530468579, Silhouette Score: 0.03510489289043682, DTW Variance: 2.400801734944875
Cluster Count: 6, Davies-Bouldin Score: 1.8730964767742802, Silhouette Score: 0.042810982720721126, DTW Variance: 1.9605766620467513
Cluster Count: 7, Davies-Bouldin Score: 1.7001912088213396, Silhouette Score: 0.03248476338359611, DTW Variance: 1.8995520400365487
Cluster Count: 8, Davies-Bouldin Score: 1.4100285371110721, Silhouette Score: 0.04459071939390642, DTW Variance: 1.36431969986789
Cluster Count: 9, Davies-Bouldin Score: 1.5757544968525115, Silhouette Score: 0.0